In [23]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [24]:
IMAGE_SIZE = (64, 64)
LEARNING_RATE = 0.01
EPOCHS = 100
BATCH_SIZE = 32
L2_LAMBDA = 0.01
DROPOUT_RATE = 0.5

In [25]:
import gdown
import zipfile

file_id = '1EF25ppO6mM_0eSlWzDrxszLt0Mmyr-wm'
output_file = 'Train.zip'

download_url = f'https://drive.google.com/uc?id={file_id}'

gdown.download(download_url, output_file, quiet=False)

with zipfile.ZipFile(output_file, 'r') as zip_ref:
    zip_ref.extractall("extracted_files/")

Downloading...
From (original): https://drive.google.com/uc?id=1EF25ppO6mM_0eSlWzDrxszLt0Mmyr-wm
From (redirected): https://drive.google.com/uc?id=1EF25ppO6mM_0eSlWzDrxszLt0Mmyr-wm&confirm=t&uuid=dabaf1d3-0461-44e6-94f4-ed133196f478
To: /content/Train.zip
100%|██████████| 2.16M/2.16M [00:00<00:00, 159MB/s]


In [26]:
import gdown
import zipfile

file_id = '10u97s_c2-ougNHQZMAP1jWvPZmWGIx1y'
output_file = 'Test.zip'

download_url = f'https://drive.google.com/uc?id={file_id}'

gdown.download(download_url, output_file, quiet=False)

with zipfile.ZipFile(output_file, 'r') as zip_ref:
    zip_ref.extractall("extracted_files/")

Downloading...
From: https://drive.google.com/uc?id=10u97s_c2-ougNHQZMAP1jWvPZmWGIx1y
To: /content/Test.zip
100%|██████████| 542k/542k [00:00<00:00, 89.0MB/s]


In [27]:
train = 'extracted_files/Train'
test= 'extracted_files/Test'

In [28]:
def preprocess_images(base_dir, classes, image_size=IMAGE_SIZE):
    """Preprocess images into arrays and return data with labels."""
    data, labels = [], []
    class_to_idx = {class_name: idx for idx, class_name in enumerate(classes)}
    for class_name in classes:
        class_dir = os.path.join(base_dir, class_name)
        image_files = os.listdir(class_dir)
        for image_file in image_files:
            image_path = os.path.join(class_dir, image_file)
            with Image.open(image_path) as img:
                img_resized = img.resize(image_size).convert('RGB')
                data.append(np.array(img_resized) / 255.0)
                labels.append(class_to_idx[class_name])
    return np.array(data), np.array(labels), class_to_idx

In [29]:
def relu(Z):
    return np.maximum(0, Z)

In [30]:
def relu_derivative(Z):
    return (Z > 0).astype(float)

In [31]:
def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return expZ / np.sum(expZ, axis=1, keepdims=True)

In [32]:
def initialize_weights(input_size, hidden_size, output_size):
    np.random.seed(42)
    W1 = np.random.randn(input_size, hidden_size) * 0.01
    b1 = np.zeros((1, hidden_size))
    W2 = np.random.randn(hidden_size, output_size) * 0.01
    b2 = np.zeros((1, output_size))
    return W1, b1, W2, b2

In [33]:
def forward_propagation(X, W1, b1, W2, b2, dropout_mask=None):
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    if dropout_mask is not None:
        A1 *= dropout_mask
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    cache = (Z1, A1, Z2, A2, dropout_mask)
    return A2, cache

In [34]:
def compute_loss(Y, A2, W1, W2, l2_lambda):
    m = Y.shape[0]
    log_likelihood = -np.log(A2[range(m), Y])
    loss = np.sum(log_likelihood) / m
    l2_penalty = (l2_lambda / (2 * m)) * (np.sum(W1**2) + np.sum(W2**2))
    return loss + l2_penalty

In [35]:
def backward_propagation(X, Y, cache, W2, l2_lambda):
    Z1, A1, Z2, A2, dropout_mask = cache
    m = X.shape[0]
    dZ2 = A2
    dZ2[range(m), Y] -= 1
    dZ2 /= m

    dW2 = np.dot(A1.T, dZ2) + (l2_lambda / m) * W2
    db2 = np.sum(dZ2, axis=0, keepdims=True)
    dA1 = np.dot(dZ2, W2.T)
    if dropout_mask is not None:
        dA1 *= dropout_mask
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = np.dot(X.T, dZ1) + (l2_lambda / m) * W1
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    return dW1, db1, dW2, db2

In [36]:
def update_weights(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

In [37]:
def predict(X, W1, b1, W2, b2):
    A2, _ = forward_propagation(X, W1, b1, W2, b2)
    return np.argmax(A2, axis=1)

In [38]:
def train_neural_network(train_data, train_labels, test_data, test_labels, input_size, hidden_size, output_size):
    W1, b1, W2, b2 = initialize_weights(input_size, hidden_size, output_size)

    for epoch in range(EPOCHS):
        indices = np.random.permutation(train_data.shape[0])
        train_data, train_labels = train_data[indices], train_labels[indices]

        for i in range(0, train_data.shape[0], BATCH_SIZE):
            X_batch = train_data[i:i + BATCH_SIZE]
            Y_batch = train_labels[i:i + BATCH_SIZE]

            dropout_mask = (np.random.rand(X_batch.shape[0], hidden_size) > DROPOUT_RATE).astype(float)

            A2, cache = forward_propagation(X_batch, W1, b1, W2, b2, dropout_mask)
            dW1, db1, dW2, db2 = backward_propagation(X_batch, Y_batch, cache, W2, L2_LAMBDA)

            W1, b1, W2, b2 = update_weights(W1, b1, W2, b2, dW1, db1, dW2, db2, LEARNING_RATE)

        A2, _ = forward_propagation(train_data, W1, b1, W2, b2)
        loss = compute_loss(train_labels, A2, W1, W2, L2_LAMBDA)
        print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {loss:.4f}")

    predictions = predict(test_data, W1, b1, W2, b2)
    accuracy = np.mean(predictions == test_labels)*100
    print(f"Test Accuracy: {accuracy:.4f}")

In [39]:
train_classes = os.listdir(train)
test_classes = os.listdir(test)
train_data, train_labels, class_mapping = preprocess_images(train, train_classes)
test_data, test_labels, _ = preprocess_images(test, test_classes)

In [40]:
train_data = train_data.reshape(train_data.shape[0], -1)
test_data = test_data.reshape(test_data.shape[0], -1)

input_size = train_data.shape[1]
hidden_size = 64
output_size = len(train_classes)
train_neural_network(train_data, train_labels, test_data, test_labels, input_size, hidden_size, output_size)


Epoch 1/100, Loss: 1.4787
Epoch 2/100, Loss: 1.1743
Epoch 3/100, Loss: 0.9035
Epoch 4/100, Loss: 0.6778
Epoch 5/100, Loss: 0.5003
Epoch 6/100, Loss: 0.3936
Epoch 7/100, Loss: 0.3337
Epoch 8/100, Loss: 0.2806
Epoch 9/100, Loss: 0.2529
Epoch 10/100, Loss: 0.2390
Epoch 11/100, Loss: 0.2065
Epoch 12/100, Loss: 0.2039
Epoch 13/100, Loss: 0.1870
Epoch 14/100, Loss: 0.1806
Epoch 15/100, Loss: 0.1685
Epoch 16/100, Loss: 0.1656
Epoch 17/100, Loss: 0.1579
Epoch 18/100, Loss: 0.1470
Epoch 19/100, Loss: 0.1505
Epoch 20/100, Loss: 0.1380
Epoch 21/100, Loss: 0.1580
Epoch 22/100, Loss: 0.1352
Epoch 23/100, Loss: 0.1373
Epoch 24/100, Loss: 0.1308
Epoch 25/100, Loss: 0.1326
Epoch 26/100, Loss: 0.1588
Epoch 27/100, Loss: 0.1313
Epoch 28/100, Loss: 0.1210
Epoch 29/100, Loss: 0.1225
Epoch 30/100, Loss: 0.1311
Epoch 31/100, Loss: 0.1229
Epoch 32/100, Loss: 0.1161
Epoch 33/100, Loss: 0.1222
Epoch 34/100, Loss: 0.1128
Epoch 35/100, Loss: 0.1111
Epoch 36/100, Loss: 0.1102
Epoch 37/100, Loss: 0.1105
Epoch 38/1